In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

End_Date = 20250209
L30_Date = 20240111
L7_Date = 20250203
MTD_Start_Date = 20250201

#working files
master = pd.read_csv('../Files/NSM OSM/2.AS ON 10FEB2025/NSM OSM REPAT MASTER.csv')
DO = pd.read_csv('../Files/NSM OSM/2.AS ON 10FEB2025/Master_DO-20250210.csv')
Sales = pd.read_csv('../Files/NSM OSM/2.AS ON 10FEB2025/STYLEWISE SALES 11JAN-09FEB.csv')
inventory_snapshot = pd.read_csv('../Files/NSM OSM/2.AS ON 10FEB2025/Inventory_snapshot_20250210.csv')


DO = DO[ (DO['master_category'] == 'Apparel') & (DO['master_brand'] == 'HRX by Hrithik Roshan') & ((DO['business_unit'] == "Men's Sports Apparel") | 
     (DO['business_unit']=="Women's Sports Apparel" )) & ((DO['gender'] == 'Men') | (DO['gender'] == 'Women'))]
Sales = Sales[ (Sales['master_brand'] == 'HRX by Hrithik Roshan') & (Sales['master_category'] == 'Apparel') & 
                ((Sales['business_unit'] =="Men's Sports Apparel") | (Sales['business_unit'] == "Women's Sports Apparel")) ]
inventory_snapshot = inventory_snapshot[ (inventory_snapshot['master_brand'] == 'HRX by Hrithik Roshan') & (inventory_snapshot['master_category'] == 'Apparel')
                    & ((inventory_snapshot['business_unit'] == "Men's Sports Apparel") | (inventory_snapshot['business_unit'] == "Women's Sports Apparel"))]
####################################################
def tagStyleID( sid ):
    df = master[ master['style_id'] == sid ]
    if len(df) == 0:
        season_lookup_dump = DO[ DO['style_id'] == sid ]
        if len(season_lookup_dump) == 0:
            return 'OSM'
        else:
            if season_lookup_dump.iloc[0]['season_code'] != 'SS25':
                return 'OSM'
            else:
                return 'NSM'
    else:
        return df.iloc[0]['tag']
        

Sales['Tag'] = Sales['style_id'].apply( tagStyleID )
inventory_snapshot['Tag'] = inventory_snapshot['style_id'].apply( tagStyleID )
MTD_Sales = Sales[ (Sales['order_created_date'] >= MTD_Start_Date) & (Sales['order_created_date'] <= End_Date) ]
L7_Sales = Sales[ (Sales['order_created_date'] >= L7_Date) & (Sales['order_created_date'] <= End_Date) ]
####################################################
def getStyleCount(gender,tag,article):
    count = len(Sales[ (Sales['gender'] == gender)  & (Sales['Tag'] == tag) & (Sales['article_type']== article)]['style_id'].value_counts())
    return count

####################################################
DO['Tag'] = DO['style_id'].apply( tagStyleID )

def getSizeAvailibilityAllStyles(gender, tag, article):
    if tag == '-' and article == '-':
        dump = DO[ (DO['gender'] == gender) ]
    elif article == '-':
        dump = DO[ (DO['gender'] == gender) & (DO['Tag'] == tag) ]
    else:
        dump = DO[ (DO['gender'] == gender) & (DO['Tag'] == tag) & (DO['article_type'] == article )]
    len_dump = len(dump)
    if len_dump == 0:
        return 0
    else:
        avg_brokenness = dump['brokenness'].sum() / len_dump
        size_availibility = 1 - avg_brokenness
        return round(size_availibility,4)
####################################################
def getStyleWiseAvailibility(sid):
    dump = DO[ DO['style_id'] == sid ]
    if len(dump) == 0:
        return 0
    else:
        return 1-dump['brokenness'].iloc[0]
####################################################################################
Sales['Availibility'] = Sales['style_id'].apply(getStyleWiseAvailibility)
def getSizeAvailibilitySoldStyles(gender, tag, article):
    if tag == '-' and article == '-':
        dump = Sales[ (Sales['gender'] == gender)]
    elif article == '-':
        dump = Sales[ (Sales['gender'] == gender) & (Sales['Tag'] == tag) ]
    else:
        dump = Sales[ (Sales['gender'] == gender) & (Sales['Tag'] == tag) & (Sales['article_type'] == article )]
    len_dump = len(dump)
    if len_dump == 0:
        return '-'
    else:
        size_availibility = dump['Availibility'].sum() / len_dump
        return round(size_availibility,4)
####################################################################################
def getInventory(gender,tag,article):
    if tag == '-' and article == '-':
        dump = inventory_snapshot[ (inventory_snapshot['gender']==gender) ]
    elif article == '-':
        dump = inventory_snapshot[ (inventory_snapshot['gender']==gender) & (inventory_snapshot['Tag']==tag) ]
    else:        
        dump = inventory_snapshot[ (inventory_snapshot['gender']==gender) & (inventory_snapshot['Tag']==tag) & (inventory_snapshot['article_type']==article)  ]
    if len(dump) == 0:
        return 0
    else:
        inventory = dump['units'].sum()
        return inventory
####################################################################################   
def getSoldQuantity(dump,gender,tag,article):
    if tag == '-' and article == '-':
        filtered_dump = dump[(dump['gender'] == gender) ]    
    elif article =='-':
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) ]
    else:
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) & (dump['article_type'] == article)]
    if len(dump) == 0:
        return 0
    else:
        sold_quantity = filtered_dump['quantity'].sum()
        return sold_quantity
####################################################################################   
def getSoldRevenue(dump,gender,tag,article):
    if tag == '-' and article == '-':
        filtered_dump = dump[(dump['gender'] == gender) ]    
    elif article =='-':
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) ]
    else:
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) & (dump['article_type'] == article)]
    if len(dump) == 0:
        return 0
    else:
        revenue = round((filtered_dump['revenue'].sum() / 100000))
        return revenue
####################################################################################   
def getTDPercentage(dump,gender,tag,article):
    if tag == '-' and article == '-':
        filtered_dump = dump[(dump['gender'] == gender) ]    
    elif article =='-':
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) ]
    else:
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) & (dump['article_type'] == article)]
    if len(filtered_dump) == 0:
        return 0
    else:
        tmrp = filtered_dump['tot_mrp'].sum()
        rev = filtered_dump['revenue'].sum()
        discount = round(((tmrp-rev)/(tmrp)),4)
        return discount
####################################################################################   
def getAISP(dump,gender,tag,article):
    if tag == '-' and article == '-':
        filtered_dump = dump[(dump['gender'] == gender) ]    
    elif article =='-':
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) ]
    else:
        filtered_dump = dump[(dump['gender'] == gender) & (dump['Tag'] == tag) & (dump['article_type'] == article)]
    if len(filtered_dump) == 0:
        return 0
    else:
        rev = filtered_dump['revenue'].sum()
        qty = filtered_dump['quantity'].sum()
        aisp = round( rev / qty )
        return aisp
####################################################################################   
def computeDOH(inventory, ros):
    if ros == 0:
        return 0
    else:
        return round(inventory / ros)
####################################################################################   
def computeSTR( inventory , sold_qty ):
    if inventory+sold_qty == 0:
        return 0
    else:
        return round( ((sold_qty)/(sold_qty+inventory)) , 4 )
####################################################################################   
#men-style-count
m_nsm_tshirts_styles = getStyleCount('Men','NSM','Tshirts')
m_nsm_trackpants_styles = getStyleCount('Men','NSM','Track Pants')
m_nsm_shorts_styles = getStyleCount('Men','NSM','Shorts')
m_nsm_jackets_styles = getStyleCount('Men','NSM','Jackets')
m_nsm_tracksuits_styles =  getStyleCount('Men','NSM','Tracksuits')
m_nsm_sweatshirts_styles = getStyleCount('Men','NSM','Sweatshirts')
m_nsm_tights_styles =  getStyleCount('Men','NSM','Tights')
m_nsm_styles = len(Sales[ (Sales['gender'] =='Men')  & (Sales['Tag'] == 'NSM')]['style_id'].value_counts())

m_osm_tshirts_styles = getStyleCount('Men','OSM','Tshirts')
m_osm_trackpants_styles = getStyleCount('Men','OSM','Track Pants')
m_osm_shorts_styles = getStyleCount('Men','OSM','Shorts')
m_osm_sweatshirts_styles = getStyleCount('Men','OSM','Sweatshirts')
m_osm_jackets_styles = getStyleCount('Men','OSM','Jackets')
m_osm_tracksuits_styles = getStyleCount('Men','OSM','Tracksuits')
m_osm_tights_styles = getStyleCount('Men','OSM','Tights')
m_osm_styles = len(Sales[ (Sales['gender'] =='Men')  & (Sales['Tag'] == 'OSM')]['style_id'].value_counts())

m_repeatx_tshirts_styles = getStyleCount('Men','RepeatX','Tshirts')
m_repeatx_trackpants_styles = getStyleCount('Men','RepeatX','Track Pants')
m_repeatx_tights_styles = getStyleCount('Men','RepeatX','Tights')
m_repeatx_tracksuits_styles = getStyleCount('Men','RepeatX','Tracksuits')
m_repeatx_shorts_styles = getStyleCount('Men','RepeatX','Shorts')
m_repeatx_styles = len(Sales[ (Sales['gender'] =='Men')  & (Sales['Tag'] == 'RepeatX')]['style_id'].value_counts())

m_total_styles = len(Sales[ (Sales['gender'] =='Men')  ]['style_id'].value_counts())
#women-style-count
w_nsm_tshirts_styles = getStyleCount('Women','NSM','Tshirts')
w_nsm_tights_styles =  getStyleCount('Women','NSM','Tights')
w_nsm_trackpants_styles = getStyleCount('Women','NSM','Track Pants')
w_nsm_jackets_styles = getStyleCount('Women','NSM','Jackets')
w_nsm_bra_styles = getStyleCount('Women','NSM','Bra')
w_nsm_sweatshirts_styles = getStyleCount('Women','NSM','Sweatshirts')
w_nsm_shorts_styles = getStyleCount('Women','NSM','Shorts')
w_nsm_tracksuits_styles =  getStyleCount('Women','NSM','Tracksuits')
w_nsm_tops_styles =  getStyleCount('Women','NSM','Tops')
w_nsm_styles = len(Sales[ (Sales['gender'] =='Women')  & (Sales['Tag'] == 'NSM')]['style_id'].value_counts())

w_osm_tshirts_styles = getStyleCount('Women','OSM','Tshirts')
w_osm_tights_styles = getStyleCount('Women','OSM','Tights')
w_osm_bra_styles = getStyleCount('Women','OSM','Bra')
w_osm_trackpants_styles = getStyleCount('Women','OSM','Track Pants')
w_osm_shorts_styles = getStyleCount('Women','OSM','Shorts')
w_osm_sweatshirts_styles = getStyleCount('Women','OSM','Sweatshirts')
w_osm_jackets_styles = getStyleCount('Women','OSM','Jackets')
w_osm_tops_styles = getStyleCount('Women','OSM','Tops')
w_osm_tracksuits_styles = getStyleCount('Women','OSM','Tracksuits')
w_osm_styles = len(Sales[ (Sales['gender'] =='Women')  & (Sales['Tag'] == 'OSM')]['style_id'].value_counts())

w_repeatx_tshirts_styles = getStyleCount('Women','RepeatX','Tshirts')
w_repeatx_trackpants_styles = getStyleCount('Women','RepeatX','Track Pants')
w_repeatx_shorts_styles = getStyleCount('Women','RepeatX','Shorts')
w_repeatx_tights_styles = getStyleCount('Women','RepeatX','Tights')
w_repeatx_tracksuits_styles = getStyleCount('Women','RepeatX','Tracksuits')
w_repeatx_bra_styles = getStyleCount('Women','RepeatX','Bra')
w_repeatx_styles = len(Sales[ (Sales['gender'] =='Women')  & (Sales['Tag'] == 'RepeatX')]['style_id'].value_counts())

w_total_styles = len(Sales[ (Sales['gender'] =='Women')  ]['style_id'].value_counts())

styles = len(Sales['style_id'].value_counts())
####################################################################################
#men-size-availibility
m_nsm_tshirts_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','Tshirts')
m_nsm_trackpants_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','Track Pants')
m_nsm_shorts_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','Shorts')
m_nsm_jackets_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','Jackets')
m_nsm_tracksuits_size_availibility =  getSizeAvailibilityAllStyles('Men','NSM','Tracksuits')
m_nsm_sweatshirts_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','Sweatshirts')
m_nsm_tights_size_availibility =  getSizeAvailibilityAllStyles('Men','NSM','Tights')
m_nsm_size_availibility = getSizeAvailibilityAllStyles('Men','NSM','-')

m_osm_tshirts_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Tshirts')
m_osm_trackpants_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Track Pants')
m_osm_shorts_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Shorts')
m_osm_sweatshirts_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Sweatshirts')
m_osm_jackets_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Jackets')
m_osm_tracksuits_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Tracksuits')
m_osm_tights_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','Tights')
m_osm_size_availibility = getSizeAvailibilityAllStyles('Men','OSM','-')

m_repeatx_tshirts_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','Tshirts')
m_repeatx_trackpants_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','Track Pants')
m_repeatx_tights_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','Tights')
m_repeatx_tracksuits_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','Tracksuits')
m_repeatx_shorts_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','Shorts')
m_repeatx_size_availibility = getSizeAvailibilityAllStyles('Men','RepeatX','-')

m_total_size_availibility = getSizeAvailibilityAllStyles('Men','-','-')
#women-size-availibility
w_nsm_tshirts_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Tshirts')
w_nsm_tights_size_availibility =  getSizeAvailibilityAllStyles('Women','NSM','Tights')
w_nsm_trackpants_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Track Pants')
w_nsm_jackets_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Jackets')
w_nsm_bra_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Bra')
w_nsm_sweatshirts_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Sweatshirts')
w_nsm_shorts_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','Shorts')
w_nsm_tracksuits_size_availibility =  getSizeAvailibilityAllStyles('Women','NSM','Tracksuits')
w_nsm_tops_size_availibility =  getSizeAvailibilityAllStyles('Women','NSM','Tops')
w_nsm_size_availibility = getSizeAvailibilityAllStyles('Women','NSM','-')

w_osm_tshirts_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Tshirts')
w_osm_tights_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Tights')
w_osm_bra_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Bra')
w_osm_trackpants_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Track Pants')
w_osm_shorts_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Shorts')
w_osm_sweatshirts_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Sweatshirts')
w_osm_jackets_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Jackets')
w_osm_tops_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Tops')
w_osm_tracksuits_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','Tracksuits')
w_osm_size_availibility = getSizeAvailibilityAllStyles('Women','OSM','-')

w_repeatx_tshirts_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Tshirts')
w_repeatx_trackpants_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Track Pants')
w_repeatx_shorts_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Shorts')
w_repeatx_tights_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Tights')
w_repeatx_tracksuits_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Tracksuits')
w_repeatx_bra_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','Bra')
w_repeatx_size_availibility = getSizeAvailibilityAllStyles('Women','RepeatX','-')

w_total_size_availibility = getSizeAvailibilityAllStyles('Women','-','-')
####################################################################################
#men-current-inventory
m_nsm_tshirts_inventory = getInventory('Men','NSM','Tshirts')
m_nsm_trackpants_inventory = getInventory('Men','NSM','Track Pants')
m_nsm_shorts_inventory = getInventory('Men','NSM','Shorts')
m_nsm_jackets_inventory = getInventory('Men','NSM','Jackets')
m_nsm_tracksuits_inventory =  getInventory('Men','NSM','Tracksuits')
m_nsm_sweatshirts_inventory = getInventory('Men','NSM','Sweatshirts')
m_nsm_tights_inventory =  getInventory('Men','NSM','Tights')
m_nsm_inventory = getInventory('Men','NSM','-')

m_osm_tshirts_inventory = getInventory('Men','OSM','Tshirts')
m_osm_trackpants_inventory = getInventory('Men','OSM','Track Pants')
m_osm_shorts_inventory = getInventory('Men','OSM','Shorts')
m_osm_sweatshirts_inventory = getInventory('Men','OSM','Sweatshirts')
m_osm_jackets_inventory = getInventory('Men','OSM','Jackets')
m_osm_tracksuits_inventory = getInventory('Men','OSM','Tracksuits')
m_osm_tights_inventory = getInventory('Men','OSM','Tights')
m_osm_inventory = getInventory('Men','OSM','-')

m_repeatx_tshirts_inventory = getInventory('Men','RepeatX','Tshirts')
m_repeatx_trackpants_inventory = getInventory('Men','RepeatX','Track Pants')
m_repeatx_tights_inventory = getInventory('Men','RepeatX','Tights')
m_repeatx_tracksuits_inventory = getInventory('Men','RepeatX','Tracksuits')
m_repeatx_shorts_inventory = getInventory('Men','RepeatX','Shorts')
m_repeatx_inventory = getInventory('Men','RepeatX','-')

m_total_inventory = getInventory('Men','-','-')
#women-current-inventory
w_nsm_tshirts_inventory = getInventory('Women','NSM','Tshirts')
w_nsm_tights_inventory =  getInventory('Women','NSM','Tights')
w_nsm_trackpants_inventory = getInventory('Women','NSM','Track Pants')
w_nsm_jackets_inventory = getInventory('Women','NSM','Jackets')
w_nsm_bra_inventory = getInventory('Women','NSM','Bra')
w_nsm_sweatshirts_inventory = getInventory('Women','NSM','Sweatshirts')
w_nsm_shorts_inventory = getInventory('Women','NSM','Shorts')
w_nsm_tracksuits_inventory =  getInventory('Women','NSM','Tracksuits')
w_nsm_tops_inventory =  getInventory('Women','NSM','Tops')
w_nsm_inventory = getInventory('Women','NSM','-')

w_osm_tshirts_inventory = getInventory('Women','OSM','Tshirts')
w_osm_tights_inventory = getInventory('Women','OSM','Tights')
w_osm_bra_inventory = getInventory('Women','OSM','Bra')
w_osm_trackpants_inventory = getInventory('Women','OSM','Track Pants')
w_osm_shorts_inventory = getInventory('Women','OSM','Shorts')
w_osm_sweatshirts_inventory = getInventory('Women','OSM','Sweatshirts')
w_osm_jackets_inventory = getInventory('Women','OSM','Jackets')
w_osm_tops_inventory = getInventory('Women','OSM','Tops')
w_osm_tracksuits_inventory = getInventory('Women','OSM','Tracksuits')
w_osm_inventory = getInventory('Women','OSM','-')

w_repeatx_tshirts_inventory = getInventory('Women','RepeatX','Tshirts')
w_repeatx_trackpants_inventory = getInventory('Women','RepeatX','Track Pants')
w_repeatx_shorts_inventory = getInventory('Women','RepeatX','Shorts')
w_repeatx_tights_inventory = getInventory('Women','RepeatX','Tights')
w_repeatx_tracksuits_inventory = getInventory('Women','RepeatX','Tracksuits')
w_repeatx_bra_inventory = getInventory('Women','RepeatX','Bra')
w_repeatx_inventory = getInventory('Women','RepeatX','-')

w_total_inventory = getInventory('Women','-','-')

inventory = w_total_inventory + m_total_inventory
####################################################################################
#men-mtd-sold-quantity
m_nsm_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','Shorts')
m_nsm_jackets_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_sold_quantity =  getSoldQuantity(MTD_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_sold_quantity =  getSoldQuantity(MTD_Sales,'Men','NSM','Tights')
m_nsm_sold_quantity = getSoldQuantity(MTD_Sales,'Men','NSM','-')
m_osm_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Track Pants')
m_osm_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Tracksuits')
m_osm_tights_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','Tights')
m_osm_sold_quantity = getSoldQuantity(MTD_Sales,'Men','OSM','-')
m_repeatx_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','Shorts')
m_repeatx_sold_quantity = getSoldQuantity(MTD_Sales,'Men','RepeatX','-')
m_total_sold_quantity = getSoldQuantity(MTD_Sales,'Men','-','-')
#women-mtd-sold-quantity
w_nsm_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Tshirts')
w_nsm_tights_sold_quantity =  getSoldQuantity(MTD_Sales,'Women','NSM','Tights')
w_nsm_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Jackets')
w_nsm_bra_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_sold_quantity =  getSoldQuantity(MTD_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_sold_quantity =  getSoldQuantity(MTD_Sales,'Women','NSM','Tops')
w_nsm_sold_quantity = getSoldQuantity(MTD_Sales,'Women','NSM','-')
w_osm_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Tshirts')
w_osm_tights_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Tights')
w_osm_bra_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Bra')
w_osm_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Track Pants')
w_osm_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Jackets')
w_osm_tops_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Tops')
w_osm_tracksuits_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','Tracksuits')
w_osm_sold_quantity = getSoldQuantity(MTD_Sales,'Women','OSM','-')
w_repeatx_tshirts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','Bra')
w_repeatx_sold_quantity = getSoldQuantity(MTD_Sales,'Women','RepeatX','-')
w_total_sold_quantity = getSoldQuantity(MTD_Sales,'Women','-','-')
mtd_sold_quantity = w_total_sold_quantity+ m_total_sold_quantity
####################################################################################
m_nsm_tshirts_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','Shorts')
m_nsm_jackets_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_revenue =  getSoldRevenue(MTD_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_revenue =  getSoldRevenue(MTD_Sales,'Men','NSM','Tights')
m_nsm_revenue = getSoldRevenue(MTD_Sales,'Men','NSM','-')
m_osm_tshirts_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Track Pants')
m_osm_shorts_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Tracksuits')
m_osm_tights_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','Tights')
m_osm_revenue = getSoldRevenue(MTD_Sales,'Men','OSM','-')
m_repeatx_tshirts_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','Shorts')
m_repeatx_revenue = getSoldRevenue(MTD_Sales,'Men','RepeatX','-')
m_total_revenue = getSoldRevenue(MTD_Sales,'Men','-','-')
w_nsm_tshirts_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Tshirts')
w_nsm_tights_revenue =  getSoldRevenue(MTD_Sales,'Women','NSM','Tights')
w_nsm_trackpants_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Jackets')
w_nsm_bra_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_revenue =  getSoldRevenue(MTD_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_revenue =  getSoldRevenue(MTD_Sales,'Women','NSM','Tops')
w_nsm_revenue = getSoldRevenue(MTD_Sales,'Women','NSM','-')
w_osm_tshirts_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Tshirts')
w_osm_tights_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Tights')
w_osm_bra_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Bra')
w_osm_trackpants_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Track Pants')
w_osm_shorts_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Jackets')
w_osm_tops_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Tops')
w_osm_tracksuits_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','Tracksuits')
w_osm_revenue = getSoldRevenue(MTD_Sales,'Women','OSM','-')
w_repeatx_tshirts_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','Bra')
w_repeatx_revenue = getSoldRevenue(MTD_Sales,'Women','RepeatX','-')
w_total_revenue = getSoldRevenue(MTD_Sales,'Women','-','-')
mtd_revenue = w_total_revenue + m_total_revenue
####################################################################################
m_nsm_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','Shorts')
m_nsm_jackets_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l7sold_quantity =  getSoldQuantity(L7_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l7sold_quantity =  getSoldQuantity(L7_Sales,'Men','NSM','Tights')
m_nsm_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','NSM','-')
m_osm_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Track Pants')
m_osm_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Tracksuits')
m_osm_tights_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','Tights')
m_osm_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','OSM','-')
m_repeatx_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','Shorts')
m_repeatx_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','RepeatX','-')
m_total_l7sold_quantity = getSoldQuantity(L7_Sales,'Men','-','-')
w_nsm_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Tshirts')
w_nsm_tights_l7sold_quantity =  getSoldQuantity(L7_Sales,'Women','NSM','Tights')
w_nsm_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Jackets')
w_nsm_bra_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l7sold_quantity =  getSoldQuantity(L7_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l7sold_quantity =  getSoldQuantity(L7_Sales,'Women','NSM','Tops')
w_nsm_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','NSM','-')
w_osm_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Tshirts')
w_osm_tights_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Tights')
w_osm_bra_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Bra')
w_osm_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Track Pants')
w_osm_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Jackets')
w_osm_tops_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Tops')
w_osm_tracksuits_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','Tracksuits')
w_osm_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','OSM','-')
w_repeatx_tshirts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','Bra')
w_repeatx_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','RepeatX','-')
w_total_l7sold_quantity = getSoldQuantity(L7_Sales,'Women','-','-')
mtd_l7sold_quantity = w_total_l7sold_quantity+ m_total_l7sold_quantity
####################################################################################
m_nsm_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','Shorts')
m_nsm_jackets_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l7revenue =  getSoldRevenue(L7_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l7revenue =  getSoldRevenue(L7_Sales,'Men','NSM','Tights')
m_nsm_l7revenue = getSoldRevenue(L7_Sales,'Men','NSM','-')
m_osm_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Track Pants')
m_osm_shorts_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Tracksuits')
m_osm_tights_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','Tights')
m_osm_l7revenue = getSoldRevenue(L7_Sales,'Men','OSM','-')
m_repeatx_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','Shorts')
m_repeatx_l7revenue = getSoldRevenue(L7_Sales,'Men','RepeatX','-')
m_total_l7revenue = getSoldRevenue(L7_Sales,'Men','-','-')
w_nsm_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Tshirts')
w_nsm_tights_l7revenue =  getSoldRevenue(L7_Sales,'Women','NSM','Tights')
w_nsm_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Jackets')
w_nsm_bra_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l7revenue =  getSoldRevenue(L7_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l7revenue =  getSoldRevenue(L7_Sales,'Women','NSM','Tops')
w_nsm_l7revenue = getSoldRevenue(L7_Sales,'Women','NSM','-')
w_osm_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Tshirts')
w_osm_tights_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Tights')
w_osm_bra_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Bra')
w_osm_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Track Pants')
w_osm_shorts_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Jackets')
w_osm_tops_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Tops')
w_osm_tracksuits_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','Tracksuits')
w_osm_l7revenue = getSoldRevenue(L7_Sales,'Women','OSM','-')
w_repeatx_tshirts_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','Bra')
w_repeatx_l7revenue = getSoldRevenue(L7_Sales,'Women','RepeatX','-')
w_total_l7revenue = getSoldRevenue(L7_Sales,'Women','-','-')
mtd_l7revenue = w_total_l7revenue + m_total_l7revenue
####################################################################################
m_nsm_tshirts_l30revenue = getSoldRevenue(Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l30revenue = getSoldRevenue(Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l30revenue = getSoldRevenue(Sales,'Men','NSM','Shorts')
m_nsm_jackets_l30revenue = getSoldRevenue(Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l30revenue =  getSoldRevenue(Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l30revenue = getSoldRevenue(Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l30revenue =  getSoldRevenue(Sales,'Men','NSM','Tights')
m_nsm_l30revenue = getSoldRevenue(Sales,'Men','NSM','-')
m_osm_tshirts_l30revenue = getSoldRevenue(Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l30revenue = getSoldRevenue(Sales,'Men','OSM','Track Pants')
m_osm_shorts_l30revenue = getSoldRevenue(Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l30revenue = getSoldRevenue(Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l30revenue = getSoldRevenue(Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l30revenue = getSoldRevenue(Sales,'Men','OSM','Tracksuits')
m_osm_tights_l30revenue = getSoldRevenue(Sales,'Men','OSM','Tights')
m_osm_l30revenue = getSoldRevenue(Sales,'Men','OSM','-')
m_repeatx_tshirts_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','Shorts')
m_repeatx_l30revenue = getSoldRevenue(Sales,'Men','RepeatX','-')
m_total_l30revenue = getSoldRevenue(Sales,'Men','-','-')
w_nsm_tshirts_l30revenue = getSoldRevenue(Sales,'Women','NSM','Tshirts')
w_nsm_tights_l30revenue =  getSoldRevenue(Sales,'Women','NSM','Tights')
w_nsm_trackpants_l30revenue = getSoldRevenue(Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l30revenue = getSoldRevenue(Sales,'Women','NSM','Jackets')
w_nsm_bra_l30revenue = getSoldRevenue(Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l30revenue = getSoldRevenue(Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l30revenue = getSoldRevenue(Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l30revenue =  getSoldRevenue(Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l30revenue =  getSoldRevenue(Sales,'Women','NSM','Tops')
w_nsm_l30revenue = getSoldRevenue(Sales,'Women','NSM','-')
w_osm_tshirts_l30revenue = getSoldRevenue(Sales,'Women','OSM','Tshirts')
w_osm_tights_l30revenue = getSoldRevenue(Sales,'Women','OSM','Tights')
w_osm_bra_l30revenue = getSoldRevenue(Sales,'Women','OSM','Bra')
w_osm_trackpants_l30revenue = getSoldRevenue(Sales,'Women','OSM','Track Pants')
w_osm_shorts_l30revenue = getSoldRevenue(Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l30revenue = getSoldRevenue(Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l30revenue = getSoldRevenue(Sales,'Women','OSM','Jackets')
w_osm_tops_l30revenue = getSoldRevenue(Sales,'Women','OSM','Tops')
w_osm_tracksuits_l30revenue = getSoldRevenue(Sales,'Women','OSM','Tracksuits')
w_osm_l30revenue = getSoldRevenue(Sales,'Women','OSM','-')
w_repeatx_tshirts_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','Bra')
w_repeatx_l30revenue = getSoldRevenue(Sales,'Women','RepeatX','-')
w_total_l30revenue = getSoldRevenue(Sales,'Women','-','-')
mtd_l30revenue = w_total_l30revenue + m_total_l30revenue

####################################################################################
#mtd-men-discount
m_nsm_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','Shorts')
m_nsm_jackets_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_td_percentage =  getTDPercentage(MTD_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_td_percentage =  getTDPercentage(MTD_Sales,'Men','NSM','Tights')
m_nsm_td_percentage = getTDPercentage(MTD_Sales,'Men','NSM','-')
m_osm_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Track Pants')
m_osm_shorts_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Tracksuits')
m_osm_tights_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','Tights')
m_osm_td_percentage = getTDPercentage(MTD_Sales,'Men','OSM','-')
m_repeatx_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','Shorts')
m_repeatx_td_percentage = getTDPercentage(MTD_Sales,'Men','RepeatX','-')
m_total_td_percentage = getTDPercentage(MTD_Sales,'Men','-','-')
#mtd-women-discount
w_nsm_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Tshirts')
w_nsm_tights_td_percentage =  getTDPercentage(MTD_Sales,'Women','NSM','Tights')
w_nsm_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Jackets')
w_nsm_bra_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_td_percentage =  getTDPercentage(MTD_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_td_percentage =  getTDPercentage(MTD_Sales,'Women','NSM','Tops')
w_nsm_td_percentage = getTDPercentage(MTD_Sales,'Women','NSM','-')
w_osm_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Tshirts')
w_osm_tights_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Tights')
w_osm_bra_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Bra')
w_osm_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Track Pants')
w_osm_shorts_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Jackets')
w_osm_tops_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Tops')
w_osm_tracksuits_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','Tracksuits')
w_osm_td_percentage = getTDPercentage(MTD_Sales,'Women','OSM','-')
w_repeatx_tshirts_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','Bra')
w_repeatx_td_percentage = getTDPercentage(MTD_Sales,'Women','RepeatX','-')
w_total_td_percentage = getTDPercentage(MTD_Sales,'Women','-','-')

mtd_td_percentage = round(((MTD_Sales['tot_mrp'].sum() - MTD_Sales['revenue'].sum()) / MTD_Sales['tot_mrp'].sum()),4)
####################################################################################
#l7-men-discount
m_nsm_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','Shorts')
m_nsm_jackets_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l7td_percentage =  getTDPercentage(L7_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l7td_percentage =  getTDPercentage(L7_Sales,'Men','NSM','Tights')
m_nsm_l7td_percentage = getTDPercentage(L7_Sales,'Men','NSM','-')
m_osm_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Track Pants')
m_osm_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Tracksuits')
m_osm_tights_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','Tights')
m_osm_l7td_percentage = getTDPercentage(L7_Sales,'Men','OSM','-')
m_repeatx_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','Shorts')
m_repeatx_l7td_percentage = getTDPercentage(L7_Sales,'Men','RepeatX','-')
m_total_l7td_percentage = getTDPercentage(L7_Sales,'Men','-','-')
#l7-women-discount
w_nsm_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Tshirts')
w_nsm_tights_l7td_percentage =  getTDPercentage(L7_Sales,'Women','NSM','Tights')
w_nsm_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Jackets')
w_nsm_bra_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l7td_percentage =  getTDPercentage(L7_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l7td_percentage =  getTDPercentage(L7_Sales,'Women','NSM','Tops')
w_nsm_l7td_percentage = getTDPercentage(L7_Sales,'Women','NSM','-')
w_osm_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Tshirts')
w_osm_tights_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Tights')
w_osm_bra_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Bra')
w_osm_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Track Pants')
w_osm_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Jackets')
w_osm_tops_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Tops')
w_osm_tracksuits_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','Tracksuits')
w_osm_l7td_percentage = getTDPercentage(L7_Sales,'Women','OSM','-')
w_repeatx_tshirts_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','Bra')
w_repeatx_l7td_percentage = getTDPercentage(L7_Sales,'Women','RepeatX','-')
w_total_l7td_percentage = getTDPercentage(L7_Sales,'Women','-','-')
l7td_percentage = round(((L7_Sales['tot_mrp'].sum() - L7_Sales['revenue'].sum()) / L7_Sales['tot_mrp'].sum()),4)
####################################################################################
#mtd-men-aisp
m_nsm_tshirts_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','Shorts')
m_nsm_jackets_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_mtd_aisp =  getAISP(MTD_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_mtd_aisp =  getAISP(MTD_Sales,'Men','NSM','Tights')
m_nsm_mtd_aisp = getAISP(MTD_Sales,'Men','NSM','-')
m_osm_tshirts_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Track Pants')
m_osm_shorts_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Tracksuits')
m_osm_tights_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','Tights')
m_osm_mtd_aisp = getAISP(MTD_Sales,'Men','OSM','-')
m_repeatx_tshirts_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','Shorts')
m_repeatx_mtd_aisp = getAISP(MTD_Sales,'Men','RepeatX','-')
m_total_mtd_aisp = getAISP(MTD_Sales,'Men','-','-')
#mtd-women-aisp
w_nsm_tshirts_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Tshirts')
w_nsm_tights_mtd_aisp =  getAISP(MTD_Sales,'Women','NSM','Tights')
w_nsm_trackpants_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Jackets')
w_nsm_bra_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_mtd_aisp =  getAISP(MTD_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_mtd_aisp =  getAISP(MTD_Sales,'Women','NSM','Tops')
w_nsm_mtd_aisp = getAISP(MTD_Sales,'Women','NSM','-')
w_osm_tshirts_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Tshirts')
w_osm_tights_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Tights')
w_osm_bra_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Bra')
w_osm_trackpants_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Track Pants')
w_osm_shorts_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Jackets')
w_osm_tops_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Tops')
w_osm_tracksuits_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','Tracksuits')
w_osm_mtd_aisp = getAISP(MTD_Sales,'Women','OSM','-')
w_repeatx_tshirts_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','Bra')
w_repeatx_mtd_aisp = getAISP(MTD_Sales,'Women','RepeatX','-')
w_total_mtd_aisp = getAISP(MTD_Sales,'Women','-','-')
mtd_aisp = round( MTD_Sales['revenue'].sum() / MTD_Sales['quantity'].sum())
####################################################################################
m_nsm_tshirts_l7_aisp = getAISP(L7_Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l7_aisp = getAISP(L7_Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l7_aisp = getAISP(L7_Sales,'Men','NSM','Shorts')
m_nsm_jackets_l7_aisp = getAISP(L7_Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l7_aisp =  getAISP(L7_Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l7_aisp = getAISP(L7_Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l7_aisp =  getAISP(L7_Sales,'Men','NSM','Tights')
m_nsm_l7_aisp = getAISP(L7_Sales,'Men','NSM','-')
m_osm_tshirts_l7_aisp = getAISP(L7_Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l7_aisp = getAISP(L7_Sales,'Men','OSM','Track Pants')
m_osm_shorts_l7_aisp = getAISP(L7_Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l7_aisp = getAISP(L7_Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l7_aisp = getAISP(L7_Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l7_aisp = getAISP(L7_Sales,'Men','OSM','Tracksuits')
m_osm_tights_l7_aisp = getAISP(L7_Sales,'Men','OSM','Tights')
m_osm_l7_aisp = getAISP(L7_Sales,'Men','OSM','-')
m_repeatx_tshirts_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','Shorts')
m_repeatx_l7_aisp = getAISP(L7_Sales,'Men','RepeatX','-')
m_total_l7_aisp = getAISP(L7_Sales,'Men','-','-')
w_nsm_tshirts_l7_aisp = getAISP(L7_Sales,'Women','NSM','Tshirts')
w_nsm_tights_l7_aisp =  getAISP(L7_Sales,'Women','NSM','Tights')
w_nsm_trackpants_l7_aisp = getAISP(L7_Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l7_aisp = getAISP(L7_Sales,'Women','NSM','Jackets')
w_nsm_bra_l7_aisp = getAISP(L7_Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l7_aisp = getAISP(L7_Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l7_aisp = getAISP(L7_Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l7_aisp =  getAISP(L7_Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l7_aisp =  getAISP(L7_Sales,'Women','NSM','Tops')
w_nsm_l7_aisp = getAISP(L7_Sales,'Women','NSM','-')
w_osm_tshirts_l7_aisp = getAISP(L7_Sales,'Women','OSM','Tshirts')
w_osm_tights_l7_aisp = getAISP(L7_Sales,'Women','OSM','Tights')
w_osm_bra_l7_aisp = getAISP(L7_Sales,'Women','OSM','Bra')
w_osm_trackpants_l7_aisp = getAISP(L7_Sales,'Women','OSM','Track Pants')
w_osm_shorts_l7_aisp = getAISP(L7_Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l7_aisp = getAISP(L7_Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l7_aisp = getAISP(L7_Sales,'Women','OSM','Jackets')
w_osm_tops_l7_aisp = getAISP(L7_Sales,'Women','OSM','Tops')
w_osm_tracksuits_l7_aisp = getAISP(L7_Sales,'Women','OSM','Tracksuits')
w_osm_l7_aisp = getAISP(L7_Sales,'Women','OSM','-')
w_repeatx_tshirts_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','Bra')
w_repeatx_l7_aisp = getAISP(L7_Sales,'Women','RepeatX','-')
w_total_l7_aisp = getAISP(L7_Sales,'Women','-','-')
l7_aisp = round( L7_Sales['revenue'].sum() / L7_Sales['quantity'].sum())
####################################################################################
#men-l30-sold-quantity
m_nsm_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','Tshirts')
m_nsm_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','Track Pants')
m_nsm_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','Shorts')
m_nsm_jackets_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','Jackets')
m_nsm_tracksuits_l30_sold_quantity =  getSoldQuantity(Sales,'Men','NSM','Tracksuits')
m_nsm_sweatshirts_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','Sweatshirts')
m_nsm_tights_l30_sold_quantity =  getSoldQuantity(Sales,'Men','NSM','Tights')
m_nsm_l30_sold_quantity = getSoldQuantity(Sales,'Men','NSM','-')
m_osm_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Tshirts')
m_osm_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Track Pants')
m_osm_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Shorts')
m_osm_sweatshirts_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Sweatshirts')
m_osm_jackets_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Jackets')
m_osm_tracksuits_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Tracksuits')
m_osm_tights_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','Tights')
m_osm_l30_sold_quantity = getSoldQuantity(Sales,'Men','OSM','-')
m_repeatx_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','Tshirts')
m_repeatx_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','Track Pants')
m_repeatx_tights_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','Tights')
m_repeatx_tracksuits_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','Tracksuits')
m_repeatx_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','Shorts')
m_repeatx_l30_sold_quantity = getSoldQuantity(Sales,'Men','RepeatX','-')
m_total_l30_sold_quantity = getSoldQuantity(Sales,'Men','-','-')
#women-l30-sold-quantity
w_nsm_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Tshirts')
w_nsm_tights_l30_sold_quantity =  getSoldQuantity(Sales,'Women','NSM','Tights')
w_nsm_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Track Pants')
w_nsm_jackets_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Jackets')
w_nsm_bra_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Bra')
w_nsm_sweatshirts_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Sweatshirts')
w_nsm_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','Shorts')
w_nsm_tracksuits_l30_sold_quantity =  getSoldQuantity(Sales,'Women','NSM','Tracksuits')
w_nsm_tops_l30_sold_quantity =  getSoldQuantity(Sales,'Women','NSM','Tops')
w_nsm_l30_sold_quantity = getSoldQuantity(Sales,'Women','NSM','-')
w_osm_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Tshirts')
w_osm_tights_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Tights')
w_osm_bra_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Bra')
w_osm_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Track Pants')
w_osm_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Shorts')
w_osm_sweatshirts_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Sweatshirts')
w_osm_jackets_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Jackets')
w_osm_tops_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Tops')
w_osm_tracksuits_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','Tracksuits')
w_osm_l30_sold_quantity = getSoldQuantity(Sales,'Women','OSM','-')
w_repeatx_tshirts_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Tshirts')
w_repeatx_trackpants_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Track Pants')
w_repeatx_shorts_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Shorts')
w_repeatx_tights_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Tights')
w_repeatx_tracksuits_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Tracksuits')
w_repeatx_bra_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','Bra')
w_repeatx_l30_sold_quantity = getSoldQuantity(Sales,'Women','RepeatX','-')
w_total_l30_sold_quantity = getSoldQuantity(Sales,'Women','-','-')
l30_sold_quantity = w_total_l30_sold_quantity+ m_total_l30_sold_quantity
####################################################################################
m_nsm_tshirts_inventory_mix=round((m_nsm_tshirts_inventory/m_nsm_inventory),4)
m_nsm_trackpants_inventory_mix=round((m_nsm_trackpants_inventory/m_nsm_inventory),4)
m_nsm_shorts_inventory_mix=round((m_nsm_shorts_inventory/m_nsm_inventory),4)
m_nsm_jackets_inventory_mix=round((m_nsm_jackets_inventory/m_nsm_inventory),4)
m_nsm_tracksuits_inventory_mix=round((m_nsm_tracksuits_inventory/m_nsm_inventory),4)
m_nsm_sweatshirts_inventory_mix=round((m_nsm_sweatshirts_inventory/m_nsm_inventory),4)
m_nsm_tights_inventory_mix=round((m_nsm_tights_inventory/m_nsm_inventory),4)
m_nsm_inventory_mix=round((m_nsm_inventory/m_total_inventory),4)
m_osm_tshirts_inventory_mix=round((m_osm_tshirts_inventory/m_osm_inventory),4)
m_osm_trackpants_inventory_mix=round((m_osm_trackpants_inventory/m_osm_inventory),4)
m_osm_shorts_inventory_mix=round((m_osm_shorts_inventory/m_osm_inventory),4)
m_osm_sweatshirts_inventory_mix=round((m_osm_sweatshirts_inventory/m_osm_inventory),4)
m_osm_jackets_inventory_mix=round((m_osm_jackets_inventory/m_osm_inventory),4)
m_osm_tracksuits_inventory_mix=round((m_osm_tracksuits_inventory/m_osm_inventory),4)
m_osm_tights_inventory_mix=round((m_osm_tights_inventory/m_osm_inventory),4)
m_osm_inventory_mix=round((m_osm_inventory/m_total_inventory),4)
m_repeatx_tshirts_inventory_mix=round((m_repeatx_tshirts_inventory/m_repeatx_inventory),4)
m_repeatx_trackpants_inventory_mix=round((m_repeatx_trackpants_inventory/m_repeatx_inventory),4)
m_repeatx_tights_inventory_mix=round((m_repeatx_tights_inventory/m_repeatx_inventory),4)
m_repeatx_tracksuits_inventory_mix=round((m_repeatx_tracksuits_inventory/m_repeatx_inventory),4)
m_repeatx_shorts_inventory_mix=round((m_repeatx_shorts_inventory/m_repeatx_inventory),4)
m_repeatx_inventory_mix=round((m_repeatx_inventory/m_total_inventory),4)
m_total_inventory_mix=round((m_total_inventory/inventory),4)
w_nsm_tshirts_inventory_mix=round((w_nsm_tshirts_inventory/w_nsm_inventory),4)
w_nsm_tights_inventory_mix=round((w_nsm_tights_inventory/w_nsm_inventory),4)
w_nsm_trackpants_inventory_mix=round((w_nsm_trackpants_inventory/w_nsm_inventory),4)
w_nsm_jackets_inventory_mix=round((w_nsm_jackets_inventory/w_nsm_inventory),4)
w_nsm_bra_inventory_mix=round((w_nsm_bra_inventory/w_nsm_inventory),4)
w_nsm_sweatshirts_inventory_mix=round((w_nsm_sweatshirts_inventory/w_nsm_inventory),4)
w_nsm_shorts_inventory_mix=round((w_nsm_shorts_inventory/w_nsm_inventory),4)
w_nsm_tracksuits_inventory_mix=round((w_nsm_tracksuits_inventory/w_nsm_inventory),4)
w_nsm_tops_inventory_mix=round((w_nsm_tops_inventory/w_nsm_inventory),4)
w_nsm_inventory_mix=round((w_nsm_inventory/w_total_inventory),4)
w_osm_tshirts_inventory_mix=round((w_osm_tshirts_inventory/w_osm_inventory),4)
w_osm_tights_inventory_mix=round((w_osm_tights_inventory/w_osm_inventory),4)
w_osm_bra_inventory_mix=round((w_osm_bra_inventory/w_osm_inventory),4)
w_osm_trackpants_inventory_mix=round((w_osm_trackpants_inventory/w_osm_inventory),4)
w_osm_shorts_inventory_mix=round((w_osm_shorts_inventory/w_osm_inventory),4)
w_osm_sweatshirts_inventory_mix=round((w_osm_sweatshirts_inventory/w_osm_inventory),4)
w_osm_jackets_inventory_mix=round((w_osm_jackets_inventory/w_osm_inventory),4)
w_osm_tops_inventory_mix=round((w_osm_tops_inventory/w_osm_inventory),4)
w_osm_tracksuits_inventory_mix=round((w_osm_tracksuits_inventory/w_osm_inventory),4)
w_osm_inventory_mix=round((w_osm_inventory/w_total_inventory),4)
w_repeatx_tshirts_inventory_mix=round((w_repeatx_tshirts_inventory/w_repeatx_inventory),4)
w_repeatx_trackpants_inventory_mix=round((w_repeatx_trackpants_inventory/w_repeatx_inventory),4)
w_repeatx_shorts_inventory_mix=round((w_repeatx_shorts_inventory/w_repeatx_inventory),4)
w_repeatx_tights_inventory_mix=round((w_repeatx_tights_inventory/w_repeatx_inventory),4)
w_repeatx_tracksuits_inventory_mix=round((w_repeatx_tracksuits_inventory/w_repeatx_inventory),4)
w_repeatx_bra_inventory_mix=round((w_repeatx_bra_inventory/w_repeatx_inventory),4)
w_repeatx_inventory_mix=round((w_repeatx_inventory/w_total_inventory),4)
w_total_inventory_mix=round((w_total_inventory/inventory),4)
####################################################################################
m_nsm_tshirts_mtdsold_qty_mix=round((m_nsm_tshirts_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_trackpants_mtdsold_qty_mix=round((m_nsm_trackpants_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_shorts_mtdsold_qty_mix=round((m_nsm_shorts_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_jackets_mtdsold_qty_mix=round((m_nsm_jackets_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_tracksuits_mtdsold_qty_mix=round((m_nsm_tracksuits_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_sweatshirts_mtdsold_qty_mix=round((m_nsm_sweatshirts_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_tights_mtdsold_qty_mix=round((m_nsm_tights_sold_quantity/m_nsm_sold_quantity),4)
m_nsm_mtdsold_qty_mix=round((m_nsm_sold_quantity/m_total_sold_quantity),4)
m_osm_tshirts_mtdsold_qty_mix=round((m_osm_tshirts_sold_quantity/m_osm_sold_quantity),4)
m_osm_trackpants_mtdsold_qty_mix=round((m_osm_trackpants_sold_quantity/m_osm_sold_quantity),4)
m_osm_shorts_mtdsold_qty_mix=round((m_osm_shorts_sold_quantity/m_osm_sold_quantity),4)
m_osm_sweatshirts_mtdsold_qty_mix=round((m_osm_sweatshirts_sold_quantity/m_osm_sold_quantity),4)
m_osm_jackets_mtdsold_qty_mix=round((m_osm_jackets_sold_quantity/m_osm_sold_quantity),4)
m_osm_tracksuits_mtdsold_qty_mix=round((m_osm_tracksuits_sold_quantity/m_osm_sold_quantity),4)
m_osm_tights_mtdsold_qty_mix=round((m_osm_tights_sold_quantity/m_osm_sold_quantity),4)
m_osm_mtdsold_qty_mix=round((m_osm_sold_quantity/m_total_sold_quantity),4)
m_repeatx_tshirts_mtdsold_qty_mix=round((m_repeatx_tshirts_sold_quantity/m_repeatx_sold_quantity),4)
m_repeatx_trackpants_mtdsold_qty_mix=round((m_repeatx_trackpants_sold_quantity/m_repeatx_sold_quantity),4)
m_repeatx_tights_mtdsold_qty_mix=round((m_repeatx_tights_sold_quantity/m_repeatx_sold_quantity),4)
m_repeatx_tracksuits_mtdsold_qty_mix=round((m_repeatx_tracksuits_sold_quantity/m_repeatx_sold_quantity),4)
m_repeatx_shorts_mtdsold_qty_mix=round((m_repeatx_shorts_sold_quantity/m_repeatx_sold_quantity),4)
m_repeatx_mtdsold_qty_mix=round((m_repeatx_sold_quantity/m_total_sold_quantity),4)
m_total_mtdsold_qty_mix=round((m_total_sold_quantity/mtd_sold_quantity),4)
w_nsm_tshirts_mtdsold_qty_mix=round((w_nsm_tshirts_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_tights_mtdsold_qty_mix=round((w_nsm_tights_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_trackpants_mtdsold_qty_mix=round((w_nsm_trackpants_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_jackets_mtdsold_qty_mix=round((w_nsm_jackets_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_bra_mtdsold_qty_mix=round((w_nsm_bra_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_sweatshirts_mtdsold_qty_mix=round((w_nsm_sweatshirts_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_shorts_mtdsold_qty_mix=round((w_nsm_shorts_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_tracksuits_mtdsold_qty_mix=round((w_nsm_tracksuits_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_tops_mtdsold_qty_mix=round((w_nsm_tops_sold_quantity/w_nsm_sold_quantity),4)
w_nsm_mtdsold_qty_mix=round((w_nsm_sold_quantity/w_total_sold_quantity),4)
w_osm_tshirts_mtdsold_qty_mix=round((w_osm_tshirts_sold_quantity/w_osm_sold_quantity),4)
w_osm_tights_mtdsold_qty_mix=round((w_osm_tights_sold_quantity/w_osm_sold_quantity),4)
w_osm_bra_mtdsold_qty_mix=round((w_osm_bra_sold_quantity/w_osm_sold_quantity),4)
w_osm_trackpants_mtdsold_qty_mix=round((w_osm_trackpants_sold_quantity/w_osm_sold_quantity),4)
w_osm_shorts_mtdsold_qty_mix=round((w_osm_shorts_sold_quantity/w_osm_sold_quantity),4)
w_osm_sweatshirts_mtdsold_qty_mix=round((w_osm_sweatshirts_sold_quantity/w_osm_sold_quantity),4)
w_osm_jackets_mtdsold_qty_mix=round((w_osm_jackets_sold_quantity/w_osm_sold_quantity),4)
w_osm_tops_mtdsold_qty_mix=round((w_osm_tops_sold_quantity/w_osm_sold_quantity),4)
w_osm_tracksuits_mtdsold_qty_mix=round((w_osm_tracksuits_sold_quantity/w_osm_sold_quantity),4)
w_osm_mtdsold_qty_mix=round((w_osm_sold_quantity/w_total_sold_quantity),4)
w_repeatx_tshirts_mtdsold_qty_mix=round((w_repeatx_tshirts_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_trackpants_mtdsold_qty_mix=round((w_repeatx_trackpants_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_shorts_mtdsold_qty_mix=round((w_repeatx_shorts_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_tights_mtdsold_qty_mix=round((w_repeatx_tights_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_tracksuits_mtdsold_qty_mix=round((w_repeatx_tracksuits_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_bra_mtdsold_qty_mix=round((w_repeatx_bra_sold_quantity/w_repeatx_sold_quantity),4)
w_repeatx_mtdsold_qty_mix=round((w_repeatx_sold_quantity/w_total_sold_quantity),4)
w_total_mtdsold_qty_mix=round((w_total_sold_quantity/mtd_sold_quantity),4)
####################################################################################
m_nsm_tshirts_l7sold_qty_mix=round((m_nsm_tshirts_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_trackpants_l7sold_qty_mix=round((m_nsm_trackpants_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_shorts_l7sold_qty_mix=round((m_nsm_shorts_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_jackets_l7sold_qty_mix=round((m_nsm_jackets_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_tracksuits_l7sold_qty_mix=round((m_nsm_tracksuits_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_sweatshirts_l7sold_qty_mix=round((m_nsm_sweatshirts_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_tights_l7sold_qty_mix=round((m_nsm_tights_l7sold_quantity/m_nsm_l7sold_quantity),4)
m_nsm_l7sold_qty_mix=round((m_nsm_l7sold_quantity/m_total_l7sold_quantity),4)
m_osm_tshirts_l7sold_qty_mix=round((m_osm_tshirts_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_trackpants_l7sold_qty_mix=round((m_osm_trackpants_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_shorts_l7sold_qty_mix=round((m_osm_shorts_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_sweatshirts_l7sold_qty_mix=round((m_osm_sweatshirts_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_jackets_l7sold_qty_mix=round((m_osm_jackets_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_tracksuits_l7sold_qty_mix=round((m_osm_tracksuits_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_tights_l7sold_qty_mix=round((m_osm_tights_l7sold_quantity/m_osm_l7sold_quantity),4)
m_osm_l7sold_qty_mix=round((m_osm_l7sold_quantity/m_total_l7sold_quantity),4)
m_repeatx_tshirts_l7sold_qty_mix=round((m_repeatx_tshirts_l7sold_quantity/m_repeatx_l7sold_quantity),4)
m_repeatx_trackpants_l7sold_qty_mix=round((m_repeatx_trackpants_l7sold_quantity/m_repeatx_l7sold_quantity),4)
m_repeatx_tights_l7sold_qty_mix=round((m_repeatx_tights_l7sold_quantity/m_repeatx_l7sold_quantity),4)
m_repeatx_tracksuits_l7sold_qty_mix=round((m_repeatx_tracksuits_l7sold_quantity/m_repeatx_l7sold_quantity),4)
m_repeatx_shorts_l7sold_qty_mix=round((m_repeatx_shorts_l7sold_quantity/m_repeatx_l7sold_quantity),4)
m_repeatx_l7sold_qty_mix=round((m_repeatx_l7sold_quantity/m_total_l7sold_quantity),4)
m_total_l7sold_qty_mix=round((m_total_l7sold_quantity/mtd_l7sold_quantity),4)
w_nsm_tshirts_l7sold_qty_mix=round((w_nsm_tshirts_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_tights_l7sold_qty_mix=round((w_nsm_tights_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_trackpants_l7sold_qty_mix=round((w_nsm_trackpants_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_jackets_l7sold_qty_mix=round((w_nsm_jackets_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_bra_l7sold_qty_mix=round((w_nsm_bra_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_sweatshirts_l7sold_qty_mix=round((w_nsm_sweatshirts_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_shorts_l7sold_qty_mix=round((w_nsm_shorts_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_tracksuits_l7sold_qty_mix=round((w_nsm_tracksuits_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_tops_l7sold_qty_mix=round((w_nsm_tops_l7sold_quantity/w_nsm_l7sold_quantity),4)
w_nsm_l7sold_qty_mix=round((w_nsm_l7sold_quantity/w_total_l7sold_quantity),4)
w_osm_tshirts_l7sold_qty_mix=round((w_osm_tshirts_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_tights_l7sold_qty_mix=round((w_osm_tights_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_bra_l7sold_qty_mix=round((w_osm_bra_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_trackpants_l7sold_qty_mix=round((w_osm_trackpants_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_shorts_l7sold_qty_mix=round((w_osm_shorts_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_sweatshirts_l7sold_qty_mix=round((w_osm_sweatshirts_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_jackets_l7sold_qty_mix=round((w_osm_jackets_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_tops_l7sold_qty_mix=round((w_osm_tops_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_tracksuits_l7sold_qty_mix=round((w_osm_tracksuits_l7sold_quantity/w_osm_l7sold_quantity),4)
w_osm_l7sold_qty_mix=round((w_osm_l7sold_quantity/w_total_l7sold_quantity),4)
w_repeatx_tshirts_l7sold_qty_mix=round((w_repeatx_tshirts_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_trackpants_l7sold_qty_mix=round((w_repeatx_trackpants_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_shorts_l7sold_qty_mix=round((w_repeatx_shorts_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_tights_l7sold_qty_mix=round((w_repeatx_tights_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_tracksuits_l7sold_qty_mix=round((w_repeatx_tracksuits_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_bra_l7sold_qty_mix=round((w_repeatx_bra_l7sold_quantity/w_repeatx_l7sold_quantity),4)
w_repeatx_l7sold_qty_mix=round((w_repeatx_l7sold_quantity/w_total_l7sold_quantity),4)
w_total_l7sold_qty_mix=round((w_total_l7sold_quantity/mtd_l7sold_quantity),4)
####################################################################################
metrics = [[m_nsm_tshirts_styles,m_nsm_tshirts_size_availibility,m_nsm_tshirts_inventory,m_nsm_tshirts_sold_quantity,m_nsm_tshirts_revenue,m_nsm_tshirts_l7sold_quantity,m_nsm_tshirts_l7revenue,m_nsm_tshirts_td_percentage,m_nsm_tshirts_l7td_percentage,m_nsm_tshirts_mtd_aisp,m_nsm_tshirts_l7_aisp,m_nsm_tshirts_l30_sold_quantity,m_nsm_tshirts_inventory_mix,m_nsm_tshirts_mtdsold_qty_mix,m_nsm_tshirts_l7sold_qty_mix,m_nsm_tshirts_l30revenue],
           [m_nsm_trackpants_styles,m_nsm_trackpants_size_availibility,m_nsm_trackpants_inventory,m_nsm_trackpants_sold_quantity,m_nsm_trackpants_revenue,m_nsm_trackpants_l7sold_quantity,m_nsm_trackpants_l7revenue,m_nsm_trackpants_td_percentage,m_nsm_trackpants_l7td_percentage,m_nsm_trackpants_mtd_aisp,m_nsm_trackpants_l7_aisp,m_nsm_trackpants_l30_sold_quantity,m_nsm_trackpants_inventory_mix,m_nsm_trackpants_mtdsold_qty_mix,m_nsm_trackpants_l7sold_qty_mix,m_nsm_trackpants_l30revenue],
           [m_nsm_shorts_styles,m_nsm_shorts_size_availibility,m_nsm_shorts_inventory,m_nsm_shorts_sold_quantity,m_nsm_shorts_revenue,m_nsm_shorts_l7sold_quantity,m_nsm_shorts_l7revenue,m_nsm_shorts_td_percentage,m_nsm_shorts_l7td_percentage,m_nsm_shorts_mtd_aisp,m_nsm_shorts_l7_aisp,m_nsm_shorts_l30_sold_quantity,m_nsm_shorts_inventory_mix,m_nsm_shorts_mtdsold_qty_mix,m_nsm_shorts_l7sold_qty_mix,m_nsm_shorts_l30revenue],
           [m_nsm_jackets_styles,m_nsm_jackets_size_availibility,m_nsm_jackets_inventory,m_nsm_jackets_sold_quantity,m_nsm_jackets_revenue,m_nsm_jackets_l7sold_quantity,m_nsm_jackets_l7revenue,m_nsm_jackets_td_percentage,m_nsm_jackets_l7td_percentage,m_nsm_jackets_mtd_aisp,m_nsm_jackets_l7_aisp,m_nsm_jackets_l30_sold_quantity,m_nsm_jackets_inventory_mix,m_nsm_jackets_mtdsold_qty_mix,m_nsm_jackets_l7sold_qty_mix,m_nsm_jackets_l30revenue],
           [m_nsm_tracksuits_styles,m_nsm_tracksuits_size_availibility,m_nsm_tracksuits_inventory,m_nsm_tracksuits_sold_quantity,m_nsm_tracksuits_revenue,m_nsm_tracksuits_l7sold_quantity,m_nsm_tracksuits_l7revenue,m_nsm_tracksuits_td_percentage,m_nsm_tracksuits_l7td_percentage,m_nsm_tracksuits_mtd_aisp,m_nsm_tracksuits_l7_aisp,m_nsm_tracksuits_l30_sold_quantity,m_nsm_tracksuits_inventory_mix,m_nsm_tracksuits_mtdsold_qty_mix,m_nsm_tracksuits_l7sold_qty_mix,m_nsm_tracksuits_l30revenue],
           [m_nsm_sweatshirts_styles,m_nsm_sweatshirts_size_availibility,m_nsm_sweatshirts_inventory,m_nsm_sweatshirts_sold_quantity,m_nsm_sweatshirts_revenue,m_nsm_sweatshirts_l7sold_quantity,m_nsm_sweatshirts_l7revenue,m_nsm_sweatshirts_td_percentage,m_nsm_sweatshirts_l7td_percentage,m_nsm_sweatshirts_mtd_aisp,m_nsm_sweatshirts_l7_aisp,m_nsm_sweatshirts_l30_sold_quantity,m_nsm_sweatshirts_inventory_mix,m_nsm_sweatshirts_mtdsold_qty_mix,m_nsm_sweatshirts_l7sold_qty_mix,m_nsm_sweatshirts_l30revenue],
           [m_nsm_tights_styles,m_nsm_tights_size_availibility,m_nsm_tights_inventory,m_nsm_tights_sold_quantity,m_nsm_tights_revenue,m_nsm_tights_l7sold_quantity,m_nsm_tights_l7revenue,m_nsm_tights_td_percentage,m_nsm_tights_l7td_percentage,m_nsm_tights_mtd_aisp,m_nsm_tights_l7_aisp,m_nsm_tights_l30_sold_quantity,m_nsm_tights_inventory_mix,m_nsm_tights_mtdsold_qty_mix,m_nsm_tights_l7sold_qty_mix,m_nsm_tights_l30revenue],
           [m_nsm_styles,m_nsm_size_availibility,m_nsm_inventory,m_nsm_sold_quantity,m_nsm_revenue,m_nsm_l7sold_quantity,m_nsm_l7revenue,m_nsm_td_percentage,m_nsm_l7td_percentage,m_nsm_mtd_aisp,m_nsm_l7_aisp,m_nsm_l30_sold_quantity,m_nsm_inventory_mix,m_nsm_mtdsold_qty_mix,m_nsm_l7sold_qty_mix,m_nsm_l30revenue],
           [m_osm_tshirts_styles,m_osm_tshirts_size_availibility,m_osm_tshirts_inventory,m_osm_tshirts_sold_quantity,m_osm_tshirts_revenue,m_osm_tshirts_l7sold_quantity,m_osm_tshirts_l7revenue,m_osm_tshirts_td_percentage,m_osm_tshirts_l7td_percentage,m_osm_tshirts_mtd_aisp,m_osm_tshirts_l7_aisp,m_osm_tshirts_l30_sold_quantity,m_osm_tshirts_inventory_mix,m_osm_tshirts_mtdsold_qty_mix,m_osm_tshirts_l7sold_qty_mix,m_osm_tshirts_l30revenue],
           [m_osm_trackpants_styles,m_osm_trackpants_size_availibility,m_osm_trackpants_inventory,m_osm_trackpants_sold_quantity,m_osm_trackpants_revenue,m_osm_trackpants_l7sold_quantity,m_osm_trackpants_l7revenue,m_osm_trackpants_td_percentage,m_osm_trackpants_l7td_percentage,m_osm_trackpants_mtd_aisp,m_osm_trackpants_l7_aisp,m_osm_trackpants_l30_sold_quantity,m_osm_trackpants_inventory_mix,m_osm_trackpants_mtdsold_qty_mix,m_osm_trackpants_l7sold_qty_mix,m_osm_trackpants_l30revenue],
           [m_osm_shorts_styles,m_osm_shorts_size_availibility,m_osm_shorts_inventory,m_osm_shorts_sold_quantity,m_osm_shorts_revenue,m_osm_shorts_l7sold_quantity,m_osm_shorts_l7revenue,m_osm_shorts_td_percentage,m_osm_shorts_l7td_percentage,m_osm_shorts_mtd_aisp,m_osm_shorts_l7_aisp,m_osm_shorts_l30_sold_quantity,m_osm_shorts_inventory_mix,m_osm_shorts_mtdsold_qty_mix,m_osm_shorts_l7sold_qty_mix,m_osm_shorts_l30revenue],
           [m_osm_sweatshirts_styles,m_osm_sweatshirts_size_availibility,m_osm_sweatshirts_inventory,m_osm_sweatshirts_sold_quantity,m_osm_sweatshirts_revenue,m_osm_sweatshirts_l7sold_quantity,m_osm_sweatshirts_l7revenue,m_osm_sweatshirts_td_percentage,m_osm_sweatshirts_l7td_percentage,m_osm_sweatshirts_mtd_aisp,m_osm_sweatshirts_l7_aisp,m_osm_sweatshirts_l30_sold_quantity,m_osm_sweatshirts_inventory_mix,m_osm_sweatshirts_mtdsold_qty_mix,m_osm_sweatshirts_l7sold_qty_mix,m_osm_sweatshirts_l30revenue],
           [m_osm_jackets_styles,m_osm_jackets_size_availibility,m_osm_jackets_inventory,m_osm_jackets_sold_quantity,m_osm_jackets_revenue,m_osm_jackets_l7sold_quantity,m_osm_jackets_l7revenue,m_osm_jackets_td_percentage,m_osm_jackets_l7td_percentage,m_osm_jackets_mtd_aisp,m_osm_jackets_l7_aisp,m_osm_jackets_l30_sold_quantity,m_osm_jackets_inventory_mix,m_osm_jackets_mtdsold_qty_mix,m_osm_jackets_l7sold_qty_mix,m_osm_jackets_l30revenue],
           [m_osm_tracksuits_styles,m_osm_tracksuits_size_availibility,m_osm_tracksuits_inventory,m_osm_tracksuits_sold_quantity,m_osm_tracksuits_revenue,m_osm_tracksuits_l7sold_quantity,m_osm_tracksuits_l7revenue,m_osm_tracksuits_td_percentage,m_osm_tracksuits_l7td_percentage,m_osm_tracksuits_mtd_aisp,m_osm_tracksuits_l7_aisp,m_osm_tracksuits_l30_sold_quantity,m_osm_tracksuits_inventory_mix,m_osm_tracksuits_mtdsold_qty_mix,m_osm_tracksuits_l7sold_qty_mix,m_osm_tracksuits_l30revenue],
           [m_osm_tights_styles,m_osm_tights_size_availibility,m_osm_tights_inventory,m_osm_tights_sold_quantity,m_osm_tights_revenue,m_osm_tights_l7sold_quantity,m_osm_tights_l7revenue,m_osm_tights_td_percentage,m_osm_tights_l7td_percentage,m_osm_tights_mtd_aisp,m_osm_tights_l7_aisp,m_osm_tights_l30_sold_quantity,m_osm_tights_inventory_mix,m_osm_tights_mtdsold_qty_mix,m_osm_tights_l7sold_qty_mix,m_osm_tights_l30revenue],
           [m_osm_styles,m_osm_size_availibility,m_osm_inventory,m_osm_sold_quantity,m_osm_revenue,m_osm_l7sold_quantity,m_osm_l7revenue,m_osm_td_percentage,m_osm_l7td_percentage,m_osm_mtd_aisp,m_osm_l7_aisp,m_osm_l30_sold_quantity,m_osm_inventory_mix,m_osm_mtdsold_qty_mix,m_osm_l7sold_qty_mix,m_osm_l30revenue],
           [m_repeatx_tshirts_styles,m_repeatx_tshirts_size_availibility,m_repeatx_tshirts_inventory,m_repeatx_tshirts_sold_quantity,m_repeatx_tshirts_revenue,m_repeatx_tshirts_l7sold_quantity,m_repeatx_tshirts_l7revenue,m_repeatx_tshirts_td_percentage,m_repeatx_tshirts_l7td_percentage,m_repeatx_tshirts_mtd_aisp,m_repeatx_tshirts_l7_aisp,m_repeatx_tshirts_l30_sold_quantity,m_repeatx_tshirts_inventory_mix,m_repeatx_tshirts_mtdsold_qty_mix,m_repeatx_tshirts_l7sold_qty_mix,m_repeatx_tshirts_l30revenue],
           [m_repeatx_trackpants_styles,m_repeatx_trackpants_size_availibility,m_repeatx_trackpants_inventory,m_repeatx_trackpants_sold_quantity,m_repeatx_trackpants_revenue,m_repeatx_trackpants_l7sold_quantity,m_repeatx_trackpants_l7revenue,m_repeatx_trackpants_td_percentage,m_repeatx_trackpants_l7td_percentage,m_repeatx_trackpants_mtd_aisp,m_repeatx_trackpants_l7_aisp,m_repeatx_trackpants_l30_sold_quantity,m_repeatx_trackpants_inventory_mix,m_repeatx_trackpants_mtdsold_qty_mix,m_repeatx_trackpants_l7sold_qty_mix,m_repeatx_trackpants_l30revenue],
           [m_repeatx_tights_styles,m_repeatx_tights_size_availibility,m_repeatx_tights_inventory,m_repeatx_tights_sold_quantity,m_repeatx_tights_revenue,m_repeatx_tights_l7sold_quantity,m_repeatx_tights_l7revenue,m_repeatx_tights_td_percentage,m_repeatx_tights_l7td_percentage,m_repeatx_tights_mtd_aisp,m_repeatx_tights_l7_aisp,m_repeatx_tights_l30_sold_quantity,m_repeatx_tights_inventory_mix,m_repeatx_tights_mtdsold_qty_mix,m_repeatx_tights_l7sold_qty_mix,m_repeatx_tights_l30revenue],
           [m_repeatx_tracksuits_styles,m_repeatx_tracksuits_size_availibility,m_repeatx_tracksuits_inventory,m_repeatx_tracksuits_sold_quantity,m_repeatx_tracksuits_revenue,m_repeatx_tracksuits_l7sold_quantity,m_repeatx_tracksuits_l7revenue,m_repeatx_tracksuits_td_percentage,m_repeatx_tracksuits_l7td_percentage,m_repeatx_tracksuits_mtd_aisp,m_repeatx_tracksuits_l7_aisp,m_repeatx_tracksuits_l30_sold_quantity,m_repeatx_tracksuits_inventory_mix,m_repeatx_tracksuits_mtdsold_qty_mix,m_repeatx_tracksuits_l7sold_qty_mix,m_repeatx_tracksuits_l30revenue],
           [m_repeatx_shorts_styles,m_repeatx_shorts_size_availibility,m_repeatx_shorts_inventory,m_repeatx_shorts_sold_quantity,m_repeatx_shorts_revenue,m_repeatx_shorts_l7sold_quantity,m_repeatx_shorts_l7revenue,m_repeatx_shorts_td_percentage,m_repeatx_shorts_l7td_percentage,m_repeatx_shorts_mtd_aisp,m_repeatx_shorts_l7_aisp,m_repeatx_shorts_l30_sold_quantity,m_repeatx_shorts_inventory_mix,m_repeatx_shorts_mtdsold_qty_mix,m_repeatx_shorts_l7sold_qty_mix,m_repeatx_shorts_l30revenue],
           [m_repeatx_styles,m_repeatx_size_availibility,m_repeatx_inventory,m_repeatx_sold_quantity,m_repeatx_revenue,m_repeatx_l7sold_quantity,m_repeatx_l7revenue,m_repeatx_td_percentage,m_repeatx_l7td_percentage,m_repeatx_mtd_aisp,m_repeatx_l7_aisp,m_repeatx_l30_sold_quantity,m_repeatx_inventory_mix,m_repeatx_mtdsold_qty_mix,m_repeatx_l7sold_qty_mix,m_repeatx_l30revenue],
           [m_total_styles,m_total_size_availibility,m_total_inventory,m_total_sold_quantity,m_total_revenue,m_total_l7sold_quantity,m_total_l7revenue,m_total_td_percentage,m_total_l7td_percentage,m_total_mtd_aisp,m_total_l7_aisp,m_total_l30_sold_quantity,m_total_inventory_mix,m_total_mtdsold_qty_mix,m_total_l7sold_qty_mix,m_total_l30revenue],
           [w_nsm_tshirts_styles,w_nsm_tshirts_size_availibility,w_nsm_tshirts_inventory,w_nsm_tshirts_sold_quantity,w_nsm_tshirts_revenue,w_nsm_tshirts_l7sold_quantity,w_nsm_tshirts_l7revenue,w_nsm_tshirts_td_percentage,w_nsm_tshirts_l7td_percentage,w_nsm_tshirts_mtd_aisp,w_nsm_tshirts_l7_aisp,w_nsm_tshirts_l30_sold_quantity,w_nsm_tshirts_inventory_mix,w_nsm_tshirts_mtdsold_qty_mix,w_nsm_tshirts_l7sold_qty_mix,w_nsm_tshirts_l30revenue],
           [w_nsm_tights_styles,w_nsm_tights_size_availibility,w_nsm_tights_inventory,w_nsm_tights_sold_quantity,w_nsm_tights_revenue,w_nsm_tights_l7sold_quantity,w_nsm_tights_l7revenue,w_nsm_tights_td_percentage,w_nsm_tights_l7td_percentage,w_nsm_tights_mtd_aisp,w_nsm_tights_l7_aisp,w_nsm_tights_l30_sold_quantity,w_nsm_tights_inventory_mix,w_nsm_tights_mtdsold_qty_mix,w_nsm_tights_l7sold_qty_mix,w_nsm_tights_l30revenue],
           [w_nsm_trackpants_styles,w_nsm_trackpants_size_availibility,w_nsm_trackpants_inventory,w_nsm_trackpants_sold_quantity,w_nsm_trackpants_revenue,w_nsm_trackpants_l7sold_quantity,w_nsm_trackpants_l7revenue,w_nsm_trackpants_td_percentage,w_nsm_trackpants_l7td_percentage,w_nsm_trackpants_mtd_aisp,w_nsm_trackpants_l7_aisp,w_nsm_trackpants_l30_sold_quantity,w_nsm_trackpants_inventory_mix,w_nsm_trackpants_mtdsold_qty_mix,w_nsm_trackpants_l7sold_qty_mix,w_nsm_trackpants_l30revenue],
           [w_nsm_jackets_styles,w_nsm_jackets_size_availibility,w_nsm_jackets_inventory,w_nsm_jackets_sold_quantity,w_nsm_jackets_revenue,w_nsm_jackets_l7sold_quantity,w_nsm_jackets_l7revenue,w_nsm_jackets_td_percentage,w_nsm_jackets_l7td_percentage,w_nsm_jackets_mtd_aisp,w_nsm_jackets_l7_aisp,w_nsm_jackets_l30_sold_quantity,w_nsm_jackets_inventory_mix,w_nsm_jackets_mtdsold_qty_mix,w_nsm_jackets_l7sold_qty_mix,w_nsm_jackets_l30revenue],
           [w_nsm_bra_styles,w_nsm_bra_size_availibility,w_nsm_bra_inventory,w_nsm_bra_sold_quantity,w_nsm_bra_revenue,w_nsm_bra_l7sold_quantity,w_nsm_bra_l7revenue,w_nsm_bra_td_percentage,w_nsm_bra_l7td_percentage,w_nsm_bra_mtd_aisp,w_nsm_bra_l7_aisp,w_nsm_bra_l30_sold_quantity,w_nsm_bra_inventory_mix,w_nsm_bra_mtdsold_qty_mix,w_nsm_bra_l7sold_qty_mix,w_nsm_bra_l30revenue],
           [w_nsm_sweatshirts_styles,w_nsm_sweatshirts_size_availibility,w_nsm_sweatshirts_inventory,w_nsm_sweatshirts_sold_quantity,w_nsm_sweatshirts_revenue,w_nsm_sweatshirts_l7sold_quantity,w_nsm_sweatshirts_l7revenue,w_nsm_sweatshirts_td_percentage,w_nsm_sweatshirts_l7td_percentage,w_nsm_sweatshirts_mtd_aisp,w_nsm_sweatshirts_l7_aisp,w_nsm_sweatshirts_l30_sold_quantity,w_nsm_sweatshirts_inventory_mix,w_nsm_sweatshirts_mtdsold_qty_mix,w_nsm_sweatshirts_l7sold_qty_mix,w_nsm_sweatshirts_l30revenue],
           [w_nsm_shorts_styles,w_nsm_shorts_size_availibility,w_nsm_shorts_inventory,w_nsm_shorts_sold_quantity,w_nsm_shorts_revenue,w_nsm_shorts_l7sold_quantity,w_nsm_shorts_l7revenue,w_nsm_shorts_td_percentage,w_nsm_shorts_l7td_percentage,w_nsm_shorts_mtd_aisp,w_nsm_shorts_l7_aisp,w_nsm_shorts_l30_sold_quantity,w_nsm_shorts_inventory_mix,w_nsm_shorts_mtdsold_qty_mix,w_nsm_shorts_l7sold_qty_mix,w_nsm_shorts_l30revenue],
           [w_nsm_tracksuits_styles,w_nsm_tracksuits_size_availibility,w_nsm_tracksuits_inventory,w_nsm_tracksuits_sold_quantity,w_nsm_tracksuits_revenue,w_nsm_tracksuits_l7sold_quantity,w_nsm_tracksuits_l7revenue,w_nsm_tracksuits_td_percentage,w_nsm_tracksuits_l7td_percentage,w_nsm_tracksuits_mtd_aisp,w_nsm_tracksuits_l7_aisp,w_nsm_tracksuits_l30_sold_quantity,w_nsm_tracksuits_inventory_mix,w_nsm_tracksuits_mtdsold_qty_mix,w_nsm_tracksuits_l7sold_qty_mix,w_nsm_tracksuits_l30revenue],
           [w_nsm_tops_styles,w_nsm_tops_size_availibility,w_nsm_tops_inventory,w_nsm_tops_sold_quantity,w_nsm_tops_revenue,w_nsm_tops_l7sold_quantity,w_nsm_tops_l7revenue,w_nsm_tops_td_percentage,w_nsm_tops_l7td_percentage,w_nsm_tops_mtd_aisp,w_nsm_tops_l7_aisp,w_nsm_tops_l30_sold_quantity,w_nsm_tops_inventory_mix,w_nsm_tops_mtdsold_qty_mix,w_nsm_tops_l7sold_qty_mix,w_nsm_tops_l30revenue],
           [w_nsm_styles,w_nsm_size_availibility,w_nsm_inventory,w_nsm_sold_quantity,w_nsm_revenue,w_nsm_l7sold_quantity,w_nsm_l7revenue,w_nsm_td_percentage,w_nsm_l7td_percentage,w_nsm_mtd_aisp,w_nsm_l7_aisp,w_nsm_l30_sold_quantity,w_nsm_inventory_mix,w_nsm_mtdsold_qty_mix,w_nsm_l7sold_qty_mix,w_nsm_l30revenue],
           [w_osm_tshirts_styles,w_osm_tshirts_size_availibility,w_osm_tshirts_inventory,w_osm_tshirts_sold_quantity,w_osm_tshirts_revenue,w_osm_tshirts_l7sold_quantity,w_osm_tshirts_l7revenue,w_osm_tshirts_td_percentage,w_osm_tshirts_l7td_percentage,w_osm_tshirts_mtd_aisp,w_osm_tshirts_l7_aisp,w_osm_tshirts_l30_sold_quantity,w_osm_tshirts_inventory_mix,w_osm_tshirts_mtdsold_qty_mix,w_osm_tshirts_l7sold_qty_mix,w_osm_tshirts_l30revenue],
           [w_osm_tights_styles,w_osm_tights_size_availibility,w_osm_tights_inventory,w_osm_tights_sold_quantity,w_osm_tights_revenue,w_osm_tights_l7sold_quantity,w_osm_tights_l7revenue,w_osm_tights_td_percentage,w_osm_tights_l7td_percentage,w_osm_tights_mtd_aisp,w_osm_tights_l7_aisp,w_osm_tights_l30_sold_quantity,w_osm_tights_inventory_mix,w_osm_tights_mtdsold_qty_mix,w_osm_tights_l7sold_qty_mix,w_osm_tights_l30revenue],
           [w_osm_bra_styles,w_osm_bra_size_availibility,w_osm_bra_inventory,w_osm_bra_sold_quantity,w_osm_bra_revenue,w_osm_bra_l7sold_quantity,w_osm_bra_l7revenue,w_osm_bra_td_percentage,w_osm_bra_l7td_percentage,w_osm_bra_mtd_aisp,w_osm_bra_l7_aisp,w_osm_bra_l30_sold_quantity,w_osm_bra_inventory_mix,w_osm_bra_mtdsold_qty_mix,w_osm_bra_l7sold_qty_mix,w_osm_bra_l30revenue],
           [w_osm_trackpants_styles,w_osm_trackpants_size_availibility,w_osm_trackpants_inventory,w_osm_trackpants_sold_quantity,w_osm_trackpants_revenue,w_osm_trackpants_l7sold_quantity,w_osm_trackpants_l7revenue,w_osm_trackpants_td_percentage,w_osm_trackpants_l7td_percentage,w_osm_trackpants_mtd_aisp,w_osm_trackpants_l7_aisp,w_osm_trackpants_l30_sold_quantity,w_osm_trackpants_inventory_mix,w_osm_trackpants_mtdsold_qty_mix,w_osm_trackpants_l7sold_qty_mix,w_osm_trackpants_l30revenue],
           [w_osm_shorts_styles,w_osm_shorts_size_availibility,w_osm_shorts_inventory,w_osm_shorts_sold_quantity,w_osm_shorts_revenue,w_osm_shorts_l7sold_quantity,w_osm_shorts_l7revenue,w_osm_shorts_td_percentage,w_osm_shorts_l7td_percentage,w_osm_shorts_mtd_aisp,w_osm_shorts_l7_aisp,w_osm_shorts_l30_sold_quantity,w_osm_shorts_inventory_mix,w_osm_shorts_mtdsold_qty_mix,w_osm_shorts_l7sold_qty_mix,w_osm_shorts_l30revenue],
           [w_osm_sweatshirts_styles,w_osm_sweatshirts_size_availibility,w_osm_sweatshirts_inventory,w_osm_sweatshirts_sold_quantity,w_osm_sweatshirts_revenue,w_osm_sweatshirts_l7sold_quantity,w_osm_sweatshirts_l7revenue,w_osm_sweatshirts_td_percentage,w_osm_sweatshirts_l7td_percentage,w_osm_sweatshirts_mtd_aisp,w_osm_sweatshirts_l7_aisp,w_osm_sweatshirts_l30_sold_quantity,w_osm_sweatshirts_inventory_mix,w_osm_sweatshirts_mtdsold_qty_mix,w_osm_sweatshirts_l7sold_qty_mix,w_osm_sweatshirts_l30revenue],
           [w_osm_jackets_styles,w_osm_jackets_size_availibility,w_osm_jackets_inventory,w_osm_jackets_sold_quantity,w_osm_jackets_revenue,w_osm_jackets_l7sold_quantity,w_osm_jackets_l7revenue,w_osm_jackets_td_percentage,w_osm_jackets_l7td_percentage,w_osm_jackets_mtd_aisp,w_osm_jackets_l7_aisp,w_osm_jackets_l30_sold_quantity,w_osm_jackets_inventory_mix,w_osm_jackets_mtdsold_qty_mix,w_osm_jackets_l7sold_qty_mix,w_osm_jackets_l30revenue],
           [w_osm_tops_styles,w_osm_tops_size_availibility,w_osm_tops_inventory,w_osm_tops_sold_quantity,w_osm_tops_revenue,w_osm_tops_l7sold_quantity,w_osm_tops_l7revenue,w_osm_tops_td_percentage,w_osm_tops_l7td_percentage,w_osm_tops_mtd_aisp,w_osm_tops_l7_aisp,w_osm_tops_l30_sold_quantity,w_osm_tops_inventory_mix,w_osm_tops_mtdsold_qty_mix,w_osm_tops_l7sold_qty_mix,w_osm_tops_l30revenue],
           [w_osm_tracksuits_styles,w_osm_tracksuits_size_availibility,w_osm_tracksuits_inventory,w_osm_tracksuits_sold_quantity,w_osm_tracksuits_revenue,w_osm_tracksuits_l7sold_quantity,w_osm_tracksuits_l7revenue,w_osm_tracksuits_td_percentage,w_osm_tracksuits_l7td_percentage,w_osm_tracksuits_mtd_aisp,w_osm_tracksuits_l7_aisp,w_osm_tracksuits_l30_sold_quantity,w_osm_tracksuits_inventory_mix,w_osm_tracksuits_mtdsold_qty_mix,w_osm_tracksuits_l7sold_qty_mix,w_osm_tracksuits_l30revenue],
           [w_osm_styles,w_osm_size_availibility,w_osm_inventory,w_osm_sold_quantity,w_osm_revenue,w_osm_l7sold_quantity,w_osm_l7revenue,w_osm_td_percentage,w_osm_l7td_percentage,w_osm_mtd_aisp,w_osm_l7_aisp,w_osm_l30_sold_quantity,w_osm_inventory_mix,w_osm_mtdsold_qty_mix,w_osm_l7sold_qty_mix,w_osm_l30revenue],
           [w_repeatx_tshirts_styles,w_repeatx_tshirts_size_availibility,w_repeatx_tshirts_inventory,w_repeatx_tshirts_sold_quantity,w_repeatx_tshirts_revenue,w_repeatx_tshirts_l7sold_quantity,w_repeatx_tshirts_l7revenue,w_repeatx_tshirts_td_percentage,w_repeatx_tshirts_l7td_percentage,w_repeatx_tshirts_mtd_aisp,w_repeatx_tshirts_l7_aisp,w_repeatx_tshirts_l30_sold_quantity,w_repeatx_tshirts_inventory_mix,w_repeatx_tshirts_mtdsold_qty_mix,w_repeatx_tshirts_l7sold_qty_mix,w_repeatx_tshirts_l30revenue],
           [w_repeatx_trackpants_styles,w_repeatx_trackpants_size_availibility,w_repeatx_trackpants_inventory,w_repeatx_trackpants_sold_quantity,w_repeatx_trackpants_revenue,w_repeatx_trackpants_l7sold_quantity,w_repeatx_trackpants_l7revenue,w_repeatx_trackpants_td_percentage,w_repeatx_trackpants_l7td_percentage,w_repeatx_trackpants_mtd_aisp,w_repeatx_trackpants_l7_aisp,w_repeatx_trackpants_l30_sold_quantity,w_repeatx_trackpants_inventory_mix,w_repeatx_trackpants_mtdsold_qty_mix,w_repeatx_trackpants_l7sold_qty_mix,w_repeatx_trackpants_l30revenue],
           [w_repeatx_shorts_styles,w_repeatx_shorts_size_availibility,w_repeatx_shorts_inventory,w_repeatx_shorts_sold_quantity,w_repeatx_shorts_revenue,w_repeatx_shorts_l7sold_quantity,w_repeatx_shorts_l7revenue,w_repeatx_shorts_td_percentage,w_repeatx_shorts_l7td_percentage,w_repeatx_shorts_mtd_aisp,w_repeatx_shorts_l7_aisp,w_repeatx_shorts_l30_sold_quantity,w_repeatx_shorts_inventory_mix,w_repeatx_shorts_mtdsold_qty_mix,w_repeatx_shorts_l7sold_qty_mix,w_repeatx_shorts_l30revenue],
           [w_repeatx_tights_styles,w_repeatx_tights_size_availibility,w_repeatx_tights_inventory,w_repeatx_tights_sold_quantity,w_repeatx_tights_revenue,w_repeatx_tights_l7sold_quantity,w_repeatx_tights_l7revenue,w_repeatx_tights_td_percentage,w_repeatx_tights_l7td_percentage,w_repeatx_tights_mtd_aisp,w_repeatx_tights_l7_aisp,w_repeatx_tights_l30_sold_quantity,w_repeatx_tights_inventory_mix,w_repeatx_tights_mtdsold_qty_mix,w_repeatx_tights_l7sold_qty_mix,w_repeatx_tights_l30revenue],
           [w_repeatx_tracksuits_styles,w_repeatx_tracksuits_size_availibility,w_repeatx_tracksuits_inventory,w_repeatx_tracksuits_sold_quantity,w_repeatx_tracksuits_revenue,w_repeatx_tracksuits_l7sold_quantity,w_repeatx_tracksuits_l7revenue,w_repeatx_tracksuits_td_percentage,w_repeatx_tracksuits_l7td_percentage,w_repeatx_tracksuits_mtd_aisp,w_repeatx_tracksuits_l7_aisp,w_repeatx_tracksuits_l30_sold_quantity,w_repeatx_tracksuits_inventory_mix,w_repeatx_tracksuits_mtdsold_qty_mix,w_repeatx_tracksuits_l7sold_qty_mix,w_repeatx_tracksuits_l30revenue],
           [w_repeatx_bra_styles,w_repeatx_bra_size_availibility,w_repeatx_bra_inventory,w_repeatx_bra_sold_quantity,w_repeatx_bra_revenue,w_repeatx_bra_l7sold_quantity,w_repeatx_bra_l7revenue,w_repeatx_bra_td_percentage,w_repeatx_bra_l7td_percentage,w_repeatx_bra_mtd_aisp,w_repeatx_bra_l7_aisp,w_repeatx_bra_l30_sold_quantity,w_repeatx_bra_inventory_mix,w_repeatx_bra_mtdsold_qty_mix,w_repeatx_bra_l7sold_qty_mix,w_repeatx_bra_l30revenue],
           [w_repeatx_styles,w_repeatx_size_availibility,w_repeatx_inventory,w_repeatx_sold_quantity,w_repeatx_revenue,w_repeatx_l7sold_quantity,w_repeatx_l7revenue,w_repeatx_td_percentage,w_repeatx_l7td_percentage,w_repeatx_mtd_aisp,w_repeatx_l7_aisp,w_repeatx_l30_sold_quantity,w_repeatx_inventory_mix,w_repeatx_mtdsold_qty_mix,w_repeatx_l7sold_qty_mix,w_repeatx_l30revenue],
           [w_total_styles,w_total_size_availibility,w_total_inventory,w_total_sold_quantity,w_total_revenue,w_total_l7sold_quantity,w_total_l7revenue,w_total_td_percentage,w_total_l7td_percentage,w_total_mtd_aisp,w_total_l7_aisp,w_total_l30_sold_quantity,w_total_inventory_mix,w_total_mtdsold_qty_mix,w_total_l7sold_qty_mix,w_total_l30revenue],
           [styles,'-',inventory,mtd_sold_quantity,mtd_revenue,mtd_l7sold_quantity,mtd_l7revenue,mtd_td_percentage,l7td_percentage,mtd_aisp,l7_aisp,l30_sold_quantity,'-','-','-',mtd_l30revenue],


]

df = pd.DataFrame(metrics,
index=['MEN-NSM-TSHIRTS','MEN-NSM-TRACKPANTS','MEN-NSM-SHORTS','MEN-NSM-JACKETS','MEN-NSM-TRACKSUITS','MEN-NSM-SWEATSHIRTS',
'MEN-NSM-TIGHTS','MEN-NSM-TOTAL','MEN-OSM-TSHIRTS','MEN-OSM-TRACKPANTS','MEN-OSM-SHORTS','MEN-OSM-SWEATSHIRTS','MEN-OSM-JACKETS',
'MEN-OSM-TRACKSUITS','MEN-OSM-TIGHTS','MEN-OSM-TOTAL','MEN-REPEATX-TSHIRTS','MEN-REPEATX-TRACKPANTS','MEN-REPEATX-TIGHTS',
'MEN-REPEATX-TRACKSUITS','MEN-REPEATX-SHORTS','MEN-REPEATX-TOTAL','MEN-TOTAL',
'WOMEN-NSM-TSHIRTS','WOMEN-NSM-TIGHTS','WOMEN-NSM-TRACKPANTS','WOMEN-NSM-JACKETS','WOMEN-NSM-BRA','WOMEN-NSM-SWEATSHIRTS',
'WOMEN-NSM-SHORTS','WOMEN-NSM-TRACKSUITS','WOMEN-NSM-TOPS','WOMEN-NSM-TOTAL','WOMEN-OSM-TSHIRTS','WOMEN-OSM-TIGHTS','WOMEN-OSM-BRA',
'WOMEN-OSM-TRACKPANTS','WOMEN-OSM-SHORTS','WOMEN-OSM-SWEATSHIRTS','WOMEN-OSM-JACKETS','WOMEN-OSM-TOPS','WOMEN-OSM-TRACKSUITS',
'WOMEN-OSM-TOTAL','WOMEN-REPEATX-TSHIRTS','WOMEN-REPEATX-TRACKPANTS','WOMEN-REPEATX-SHORTS','WOMEN-REPEATX-TIGHTS','WOMEN-REPEATX-TRACKSUITS',
'WOMEN-REPEATX-BRA','WOMEN-REPEATX-TOTAL','WOMEN-TOTAL','GRAND TOTAL'],
                  
columns=['Style','Size_Availibility','Inventory','MTD_Sold_Qty','MTD Revenue(lacs)','L7_Sold_Qty','L7 Revenue(lacs)','MTD TD%',
'L7 TD%','MTD AISP','L7 AISP','L30_Sold_Qty','Inventory_Mix','MTD_Sold_Qty_Mix','L7_Sold_Qty_Mix','L30 Revenue (lacs)'])

####################################################################################
df['L7_ROS'] = df['L7_Sold_Qty'].apply(lambda qty : round(qty/7) )
df['L30_ROS'] = df['L30_Sold_Qty'].apply(lambda qty : round(qty/30) )
df['L7_DOH'] = df.apply( lambda x : computeDOH(x.Inventory, x.L7_ROS), axis=1 )
df['L30_DOH'] = df.apply( lambda x : computeDOH(x.Inventory, x.L30_ROS), axis=1 )
df['L7_STR'] = df.apply( lambda x : computeSTR( x.Inventory , x.L7_Sold_Qty ),axis=1 )
df['L30_STR'] = df.apply( lambda x : computeSTR( x.Inventory , x.L30_Sold_Qty ),axis=1 )
####################################################################################
df.to_excel('NSM-OSM-TEMP.xlsx')